In [ ]:
%pip install pandas sqlalchemy snowflake-connector-python snowflake-sqlalchemy holidays python-dotenv

In [6]:
from os import getenv
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from dotenv import load_dotenv
import calendar
import locale
from datetime import datetime, timedelta
import holidays

In [7]:
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

In [8]:
def get_estacao(date):
    month = date.month
    day = date.day
    
    if (month == 12 and day >= 21) or month in [1, 2] or (month == 3 and day < 21):
        return 'Verão'
    elif (month == 3 and day >= 21) or month in [4, 5] or (month == 6 and day < 21):
        return 'Outono'
    elif (month == 6 and day >= 21) or month in [7, 8] or (month == 9 and day < 23):
        return 'Inverno'
    elif (month == 9 and day >= 23) or month in [10, 11] or (month == 12 and day < 21):
        return 'Primavera'

start_date = '2000-01-01'
end_date = '2030-12-31'

date_range = pd.date_range(start=start_date, end=end_date)

df_time = pd.DataFrame(date_range, columns=['data'])

df_time['dia'] = df_time['data'].dt.day
df_time['nr_mes'] = df_time['data'].dt.month
df_time['ds_mes'] = df_time['data'].dt.month.apply(lambda x: calendar.month_name[x])
df_time['nr_trimestre'] = df_time['data'].dt.quarter
df_time['estacao'] = df_time['data'].apply(get_estacao)

br_holidays = holidays.Brazil()
df_time['in_feriado'] = df_time['data'].apply(lambda x: x in br_holidays)



In [ ]:
load_dotenv()

In [ ]:
snowflake_conn_string = URL(
    account= getenv('SNOWFLAKE_ACCOUNT'),
    user= getenv('SNOWFLAKE_USER'),
    password= getenv('SNOWFLAKE_PASSWORD'),
    database= getenv('SNOWFLAKE_DATABASE')
)
snowflake = create_engine(snowflake_conn_string)

In [ ]:
df_time.to_sql(
        name='d_tempo',
        con=snowflake,
        schema='dw',
        if_exists='replace',
        index=False
    )